In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
import math

In [2]:
rates = pd.read_csv('./InterestRates.csv')
#rates['key'] = rates['pricedate'].str.cat(rates['maturity'] , sep =", ")
rates['pricedate'] = pd.to_datetime(rates.pricedate, format='%Y-%m-%d', errors='ignore')
#print (rates.dtypes)
#rates.info()
rates[['number1','month']] = rates.maturity.str.split("M",expand=True,)
rates[['number2','year']] = rates.maturity.str.split("Y",expand=True,)

#So, I want to place number with pfv where pfv is not null, how can I achieve that?
rates['number'] = np.where(~rates['year'].isnull(),rates['number2'],rates['number1'])
rates['number'] = pd.to_numeric(rates['number'])
rates['MorY'] = np.where(~rates['year'].isnull(),'Y','M')

rates = rates.drop(columns=['number1','number2','month','year'])
rates['days'] = np.where(rates['MorY']=='Y',rates['number']*360,rates['number']*30)

#rates.info()
#rates.head(100)

#for the rates I am thinking on using the smallest one, 1 month
rates_1m = rates[rates['maturity']=='1M']
#rates_1m.info()
#rates_1m.head()

#len(rates.key.unique())
#len(rates.maturity.unique())

In [3]:
prices = pd.read_csv('./PricesFile1.csv')
prices['pricedate_str'] = prices['pricedate']
prices['delivdate_str'] = prices['delivdate']
prices['key'] = prices['pricedate_str'].str.cat(prices['delivdate_str'], sep=',')

#drop other variables
prices = prices[prices['priceindex']=='WTI NYMEX LIGHT SWEET']

#fixing dates
#pricedate and delivdate
prices['pricedate'] = pd.to_datetime(prices.pricedate, format='%Y-%m-%d', errors='ignore')
prices['delivdate'] = pd.to_datetime(prices.delivdate, format='%Y-%m-%d', errors='ignore')

#calculating duration ****QUESTION Should we drop negatives according to scott answer?
prices['duration'] = prices['delivdate'] - prices['pricedate']
prices['duration_n'] = prices['duration'].dt.days
#drop negative duration  **61 elements dropped
prices = prices[prices['duration_n']>=0]

#Checking if prices are unique for same dates
#len(prices.key.unique())

#prices.priceindex.unique()
prices.info()
prices.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1999 entries, 4071 to 6129
Data columns (total 9 columns):
priceindex       1999 non-null object
pricedate        1999 non-null datetime64[ns]
delivdate        1999 non-null datetime64[ns]
price            1999 non-null float64
pricedate_str    1999 non-null object
delivdate_str    1999 non-null object
key              1999 non-null object
duration         1999 non-null timedelta64[ns]
duration_n       1999 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(1), object(4), timedelta64[ns](1)
memory usage: 156.2+ KB


,priceindex,pricedate,delivdate,price,pricedate_str,delivdate_str,key,duration,duration_n
4071,WTI NYMEX LIGHT SWEET,2020-03-02,2020-04-01,46.749,2020-03-02,2020-04-01,"2020-03-02,2020-04-01",30 days,30
4072,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,2020-03-02,2020-05-01,"2020-03-02,2020-05-01",60 days,60
4073,WTI NYMEX LIGHT SWEET,2020-03-02,2020-06-01,47.049,2020-03-02,2020-06-01,"2020-03-02,2020-06-01",91 days,91
4074,WTI NYMEX LIGHT SWEET,2020-03-02,2020-07-01,47.139,2020-03-02,2020-07-01,"2020-03-02,2020-07-01",121 days,121
4075,WTI NYMEX LIGHT SWEET,2020-03-02,2020-08-01,47.199,2020-03-02,2020-08-01,"2020-03-02,2020-08-01",152 days,152


In [10]:
wti_imvl20=pd.read_csv("./WTI_ImpliedVols2020.csv")
wti_imvl21=pd.read_csv("./WTI_ImpliedVols2021.csv")
wti_imvl22=pd.read_csv("./WTI_ImpliedVols2022.csv")

#Append
frames=[wti_imvl20,wti_imvl21,wti_imvl22]
wti_imvl = pd.concat(frames)

#generating key
wti_imvl['volatilitydate_str'] = wti_imvl['volatilitydate']
wti_imvl['begtime_str'] = wti_imvl['begtime']
wti_imvl['key'] = wti_imvl['volatilitydate_str'].str.cat(wti_imvl['begtime_str'], sep=',')

#fixing dates
#volatilitydate and begtime
wti_imvl['volatilitydate'] = pd.to_datetime(wti_imvl.volatilitydate, format='%Y-%m-%d', errors='ignore')
wti_imvl['begtime'] = pd.to_datetime(wti_imvl.begtime, format='%Y-%m-%d', errors='ignore')

#calculating duration ****QUESTION Should we drop negatives according to scott answer?
wti_imvl['duration'] = wti_imvl['begtime'] - wti_imvl['volatilitydate']
wti_imvl['duration_n'] = wti_imvl['duration'].dt.days
#wti_imvl.info()
#drop negative duration  **No elements dropped
wti_imvl = wti_imvl[wti_imvl['duration_n']>=0]
#Checking if wti_imvl are unique for same dates
len(wti_imvl.key.unique())

#TAKING MEAN VALUES because they are not unique, then I am taking mean value of strikeprice and volatility, thought I would like to ponderate by volume 
#but not volumen available       ***Ask Anatoliy
uniques_strike = wti_imvl.groupby('key')['strikeprice'].mean()
uniques_volatility = wti_imvl.groupby('key')['volatility'].mean()

#MERGE
wti_vol = pd.merge(uniques_strike,uniques_volatility,on='key',how='left')
#wti_vol.info()
#wti_vol.head()
wti_imvl_2 = wti_imvl.drop(columns=['strikeprice','volatility','volatilitydate_str','begtime_str'])
wti_imvl_2 = wti_imvl_2.drop_duplicates()

wti_imvl_2 = pd.merge(wti_vol,wti_imvl_2,on='key')

#wti_imvl.volatilityindex.unique()
wti_imvl_2.info()
#wti_imvl_2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1198 entries, 0 to 1197
Data columns (total 8 columns):
key                1198 non-null object
strikeprice        1198 non-null float64
volatility         1198 non-null float64
volatilityindex    1198 non-null object
volatilitydate     1198 non-null datetime64[ns]
begtime            1198 non-null datetime64[ns]
duration           1198 non-null timedelta64[ns]
duration_n         1198 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(1), object(2), timedelta64[ns](1)
memory usage: 84.2+ KB


In [11]:
#As I will merge volatility with prices I will drop variables I dont want

#####Volatility
#key                1198 non-null object
#strikeprice        1198 non-null float64
#volatility         1198 non-null float64
#volatilityindex    1198 non-null object
wti_imvl_2 = wti_imvl_2.drop(columns=['volatilitydate','begtime','duration','duration_n'])

######price
#priceindex       1999 non-null object
#pricedate        1999 non-null datetime64[ns]
#delivdate        1999 non-null datetime64[ns]
#price            1999 non-null float64
#key              1999 non-null object
#duration         1999 non-null timedelta64[ns]
#duration_n       1999 non-null int64
prices = prices.drop(columns=['pricedate_str','delivdate_str'])

master_wti = pd.merge(prices,wti_imvl_2,on='key')

#Merging Rates
master_wti = pd.merge(master_wti,rates_1m,on='pricedate',how='left')

#master_wti.info()
#master_wti.head()

In [12]:
#Black 76


# t = pricedate          1198 non-null datetime64[ns]
# T = delivdate          1198 non-null datetime64[ns]
# F(t,T) = price              1198 non-null float64
# T - t = duration_n         1198 non-null int64
# K = strikeprice        1198 non-null float64
# sigma = volatility         1198 non-null float64
# r = bidrate            1198 non-null float64

master_wti.rename(columns={'pricedate': 't', 'delivdate': 'T', 'price': 'F', 'duration_n': 'T_t', 'strikeprice': 'K', 'volatility': 'sigma', 'bidrate': 'r'}, inplace=True)

#FIX A Te

Te_t = 15/360

#Calculating call value
Nd1 = norm.cdf((np.log(master_wti['F']/master_wti['K']) + (1/2 * master_wti['sigma']**2 * Te_t))/(master_wti['sigma'] * np.sqrt(Te_t)))
Nd2 = norm.cdf((np.log(master_wti['F']/master_wti['K']) - (1/2 * master_wti['sigma']**2 * Te_t))/(master_wti['sigma'] * np.sqrt(Te_t)))

master_wti['call_15'] = np.exp(-master_wti['r']*Te_t)*(master_wti['F']*Nd1 - master_wti['K']*Nd2)

master_wti.info()
master_wti.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1198 entries, 0 to 1197
Data columns (total 16 columns):
priceindex         1198 non-null object
t                  1198 non-null datetime64[ns]
T                  1198 non-null datetime64[ns]
F                  1198 non-null float64
key                1198 non-null object
duration           1198 non-null timedelta64[ns]
T_t                1198 non-null int64
K                  1198 non-null float64
sigma              1198 non-null float64
volatilityindex    1198 non-null object
maturity           1198 non-null object
r                  1198 non-null float64
number             1198 non-null int64
MorY               1198 non-null object
days               1198 non-null int64
call_15            1198 non-null float64
dtypes: datetime64[ns](2), float64(5), int64(3), object(5), timedelta64[ns](1)
memory usage: 159.1+ KB


,priceindex,t,T,F,key,duration,T_t,K,sigma,volatilityindex,maturity,r,number,MorY,days,call_15
0,WTI NYMEX LIGHT SWEET,2020-03-02,2020-04-01,46.749,"2020-03-02,2020-04-01",30 days,30,45.000000,0.558333,V_WTI NYMEX LIGHT SWEET,1M,1.110001,1,M,30,2.935647
1,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,"2020-03-02,2020-05-01",60 days,60,45.000000,0.514951,V_WTI NYMEX LIGHT SWEET,1M,1.110001,1,M,30,2.898581
2,WTI NYMEX LIGHT SWEET,2020-03-02,2020-06-01,47.049,"2020-03-02,2020-06-01",91 days,91,45.000000,0.458469,V_WTI NYMEX LIGHT SWEET,1M,1.110001,1,M,30,2.800297
3,WTI NYMEX LIGHT SWEET,2020-03-02,2020-07-01,47.139,"2020-03-02,2020-07-01",121 days,121,45.000000,0.424321,V_WTI NYMEX LIGHT SWEET,1M,1.110001,1,M,30,2.753751
4,WTI NYMEX LIGHT SWEET,2020-03-02,2020-09-01,47.249,"2020-03-02,2020-09-01",183 days,183,44.985185,0.365481,V_WTI NYMEX LIGHT SWEET,1M,1.110001,1,M,30,2.664706
